# Notebook 1: Data Preparation 

### Introduction + Data Prep

Objective: Introduce topic, explain key questions, and prep data for the project.

### Step 1: Import libraries

The libraries below will allow us to examine our datasets and clean them.

In [1]:
import pandas as pd 
import geopandas as gpd

### Step 2: Load, clean, and display dataset

1. Loading dataset using pandas library
2. Clean columns + rows by standardizing names, filling in empty rows, dropping duplicate rows, etc.
3. Displaying several rows of data to see what dataset includes 

Dataset 1: Weather + Fire Day-Level Data

Daily-level data that combines weather variables (like temperature, wind speed, precipitation) with fire occurrence information.

In [2]:
df1 = pd.read_csv("CAWeatherFirePrediction.csv")
df1.columns = df1.columns.str.strip().str.lower().str.replace(" ", "_")
df1.isnull().sum()


date                      0
precipitation             1
max_temp                  1
min_temp                  1
avg_wind_speed           12
fire_start_day            0
year                      0
temp_range                1
wind_temp_ratio          12
month                     0
season                    0
lagged_precipitation      0
lagged_avg_wind_speed     0
day_of_year               0
dtype: int64

In [3]:
df1.drop_duplicates(inplace=True)

In [4]:
# For precipitation (only 1 missing, treat missing as 0 rainfall)
df1['precipitation'] = df1['precipitation'].fillna(0)

# For max and min temps (1 missing each, fill with median)
df1['max_temp'] = df1['max_temp'].fillna(df1['max_temp'].median())
df1['min_temp'] = df1['min_temp'].fillna(df1['min_temp'].median())

# For avg_wind_speed (12 missing, fill with median)
df1['avg_wind_speed'] = df1['avg_wind_speed'].fillna(df1['avg_wind_speed'].median())

# For temp_range (1 missing, recalc if possible)
df1['temp_range'] = df1['temp_range'].fillna(df1['max_temp'] - df1['min_temp'])

# For wind_temp_ratio (12 missing, recalc if possible)
df1['wind_temp_ratio'] = df1['wind_temp_ratio'].fillna(df1['avg_wind_speed'] / df1['temp_range'])

In [5]:
df1.head()

,date,precipitation,max_temp,min_temp,avg_wind_speed,fire_start_day,year,temp_range,wind_temp_ratio,month,season,lagged_precipitation,lagged_avg_wind_speed,day_of_year
0,1984-01-01,0.0,79.0,51.0,4.70,False,1984,28.0,0.059494,1,Winter,0.0,4.700,1
1,1984-01-02,0.0,71.0,46.0,5.59,False,1984,25.0,0.078732,1,Winter,0.0,5.145,2
2,1984-01-03,0.0,70.0,47.0,5.37,False,1984,23.0,0.076714,1,Winter,0.0,5.220,3
3,1984-01-04,0.0,76.0,45.0,4.70,False,1984,31.0,0.061842,1,Winter,0.0,5.090,4
4,1984-01-05,0.0,74.0,49.0,5.14,False,1984,25.0,0.069459,1,Winter,0.0,5.100,5


### Step 3: Further examine data in dataset

Looking further into data and noticing things such as:
   
   1. Data Type
   2. Data Variability

In [6]:
df1.info()
df1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14988 entries, 0 to 14987
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   14988 non-null  object 
 1   precipitation          14988 non-null  float64
 2   max_temp               14988 non-null  float64
 3   min_temp               14988 non-null  float64
 4   avg_wind_speed         14988 non-null  float64
 5   fire_start_day         14988 non-null  bool   
 6   year                   14988 non-null  int64  
 7   temp_range             14988 non-null  float64
 8   wind_temp_ratio        14988 non-null  float64
 9   month                  14988 non-null  int64  
 10  season                 14988 non-null  object 
 11  lagged_precipitation   14988 non-null  float64
 12  lagged_avg_wind_speed  14988 non-null  float64
 13  day_of_year            14988 non-null  int64  
dtypes: bool(1), float64(8), int64(3), object(2)
memory usa

,precipitation,max_temp,min_temp,avg_wind_speed,year,temp_range,wind_temp_ratio,month,lagged_precipitation,lagged_avg_wind_speed,day_of_year
count,14988.000000,14988.000000,14988.000000,14988.000000,14988.000000,14988.000000,14988.000000,14988.000000,14988.000000,14988.000000,14988.000000
mean,0.032313,70.534961,56.494129,7.434878,2004.016813,14.040833,0.107412,6.518281,0.226188,7.434198,182.992994
std,0.179538,7.263206,6.767461,2.129146,11.843342,5.995133,0.038720,3.451037,0.648705,1.387849,105.523627
min,0.000000,50.000000,33.000000,1.790000,1984.000000,2.000000,0.023553,1.000000,0.000000,3.227143,1.000000
25%,0.000000,65.000000,51.000000,6.040000,1994.000000,10.000000,0.085395,4.000000,0.000000,6.518571,92.000000
50%,0.000000,70.000000,57.000000,7.160000,2004.000000,12.000000,0.102286,7.000000,0.000000,7.478571,183.000000
75%,0.000000,75.000000,62.000000,8.500000,2014.000000,17.000000,0.120513,10.000000,0.060000,8.278571,274.000000
max,4.530000,106.000000,77.000000,26.170000,2025.000000,41.000000,1.193333,12.000000,8.180000,13.932857,366.000000


### Step 4: Testing data

Testing our data to ensure it's reliable before proceeding with it.  

In [7]:
# Check there are no missing date values
assert df1["date"].isnull().sum() == 0, "Missing values found in 'date' column"

# Check temperature ranges in °F
assert df1["max_temp"].max() <= 140, "Max temperature seems too high for Fahrenheit"
assert df1["min_temp"].min() >= -60, "Min temperature seems too low for Fahrenheit"


### Step 5: Repeat as needed for all datasets

Dataset 2: Fire Perimeter and Metadata

Incident-level data with detailed information about each wildfire event, including its size, location, dates, and cause.

In [8]:
df2 = gpd.read_file('CAFireParameter.csv')
df2.columns = df2.columns.str.strip().str.lower().str.replace(" ", "_")
df2.isnull().sum()


objectid                        0
year                            0
state                           0
agency                          0
unit_id                         0
fire_name                       0
local_incident_number           0
alarm_date                      0
containment_date                0
cause                           0
collection_method               0
management_objective            0
gis_calculated_acres            0
comments                        0
complex_name                    0
irwin_id                        0
fire_number_(historical_use)    0
complex_id                      0
decades                         0
shape__area                     0
shape__length                   0
dtype: int64

In [9]:
df2 = df2.drop_duplicates()

In [10]:
df2.head()

,objectid,year,state,agency,unit_id,fire_name,local_incident_number,alarm_date,containment_date,cause,...,management_objective,gis_calculated_acres,comments,complex_name,irwin_id,fire_number_(historical_use),complex_id,decades,shape__area,shape__length
0,1,2025,CA,CDF,LDF,PALISADES,00000738,1/7/2025 8:00:00 AM,1/31/2025 8:00:00 AM,14,...,1,23448.88,,,{A7EA5D21-F882-44B8-BF64-44AB11059DC1},,,2020-January 2025,138651835,140231.608232476
1,2,2025,CA,CDF,LAC,EATON,00009087,1/8/2025 8:00:00 AM,1/31/2025 8:00:00 AM,14,...,1,14056.26,,,{72660ADC-B5EF-4D96-A33F-B4EA3740A4E3},,,2020-January 2025,83363928.8945313,104933.207223671
2,3,2025,CA,CDF,ANF,HUGHES,00250270,1/22/2025 8:00:00 AM,1/28/2025 8:00:00 AM,14,...,1,10396.8,,,{994072D2-E154-434A-BB95-6F6C94C40829},,,2020-January 2025,62160638.9140625,96698.599857731
3,4,2025,CA,CCO,VNC,KENNETH,00003155,1/9/2025 8:00:00 AM,2/4/2025 8:00:00 AM,14,...,1,998.7378,from OES Intel 24,,{842FB37B-7AC8-4700-BB9C-028BF753D149},,,2020-January 2025,5919678.1015625,15602.0048488298
4,5,2025,CA,CDF,LDF,HURST,00003294,1/7/2025 8:00:00 AM,1/9/2025 8:00:00 AM,14,...,1,831.3855,,,{F4E810AD-CDF3-4ED4-B63F-03D43785BA7B},,,2020-January 2025,4946081.875,16094.2170731712


In [11]:
df2.info()
df2.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22810 entries, 0 to 22809
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   objectid                      22810 non-null  object
 1   year                          22810 non-null  object
 2   state                         22810 non-null  object
 3   agency                        22810 non-null  object
 4   unit_id                       22810 non-null  object
 5   fire_name                     22810 non-null  object
 6   local_incident_number         22810 non-null  object
 7   alarm_date                    22810 non-null  object
 8   containment_date              22810 non-null  object
 9   cause                         22810 non-null  object
 10  collection_method             22810 non-null  object
 11  management_objective          22810 non-null  object
 12  gis_calculated_acres          22810 non-null  object
 13  comments        

,objectid,year,state,agency,unit_id,fire_name,local_incident_number,alarm_date,containment_date,cause,...,management_objective,gis_calculated_acres,comments,complex_name,irwin_id,fire_number_(historical_use),complex_id,decades,shape__area,shape__length
count,22810,22810,22810,22810,22810,22810,22810,22810,22810,22810,...,22810,22810,22810,22810,22810,22810,22810,22810,22810,22810
unique,22810,129,4,11,111,9094,7607,8968,5493,18,...,3,22767,1749,142,3127,3173,134,9,22799,22802
top,22811,2017,CA,USF,LAC,,00000000,,,14,...,1,0.1008057,,,,,,1950-1959,14954868.8085938,21360.4481476672
freq,1,609,22794,9692,2029,6489,12470,5396,12630,10316,...,22262,3,19975,22210,19668,5610,22244,7012,2,2


In [12]:
# Check for missing values in critical columns
assert df2['year'].notnull().all(), "Missing year values"
assert df2['state'].notnull().all(), "Missing state values"
assert df2['gis_calculated_acres'].notnull().all(), "Missing acreage values"

# Check that 'alarm_date' and 'containment_date' are valid dates and no nulls
assert df2['alarm_date'].notnull().all(), "Missing alarm_date values"
assert df2['containment_date'].notnull().all(), "Missing containment_date values"

# Check for unique fire names or IDs if relevant
assert df2['fire_name'].notnull().all(), "Missing fire names"

Dataset 3: Built Environment Risk + Socioeconomic Indicators

County-level data from FEMA and other sources describing vulnerability, exposure, and capacity to respond to wildfires.

In [13]:
df3 = pd.read_csv("countyRisk.csv")
df3.columns = df3.columns.str.strip().str.lower().str.replace(" ", "_")
df3.isnull().sum()


oid_              0
nri_id            0
state             0
stateabbrv        0
statefips         0
                 ..
wntw_alr_npctl    0
wntw_riskv        0
wntw_risks        0
wntw_riskr        0
nri_ver           0
Length: 465, dtype: int64

In [14]:
df3.drop_duplicates(inplace=True)

In [15]:
df3.head()

,oid_,nri_id,state,stateabbrv,statefips,county,countytype,countyfips,stcofips,population,...,wntw_eals,wntw_ealr,wntw_alrb,wntw_alrp,wntw_alra,wntw_alr_npctl,wntw_riskv,wntw_risks,wntw_riskr,nri_ver
0,188,C06001,California,CA,6,Alameda,County,1,6001,1678284,...,4.302074,Very Low,0.000000e+00,0.000000e+00,0.000000e+00,4.271123,0.000751,1.622654,Very Low,March 2023
1,189,C06003,California,CA,6,Alpine,County,3,6003,1198,...,69.730734,Relatively Moderate,4.242441e-06,7.425749e-06,3.838500e-11,99.257196,144225.447575,72.955775,Relatively Moderate,March 2023
2,190,C06005,California,CA,6,Amador,County,5,6005,40427,...,28.938409,Relatively Low,1.145622e-07,3.837438e-08,1.245000e-12,19.777159,21401.175814,25.898823,Relatively Low,March 2023
3,191,C06007,California,CA,6,Butte,County,7,6007,211490,...,15.753637,Very Low,1.471309e-09,3.118908e-09,1.160000e-13,5.787682,9423.088265,13.203945,Very Low,March 2023
4,192,C06009,California,CA,6,Calaveras,County,9,6009,45237,...,29.743114,Relatively Low,9.796616e-08,3.500279e-08,1.511000e-12,18.848654,19805.739936,24.307986,Very Low,March 2023


In [16]:
df3.info()
df3.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Columns: 465 entries, oid_ to nri_ver
dtypes: float64(396), int64(5), object(64)
memory usage: 210.8+ KB


,oid_,statefips,countyfips,stcofips,population,buildvalue,agrivalue,area,risk_value,risk_score,...,wntw_ealpe,wntw_eala,wntw_ealt,wntw_eals,wntw_alrb,wntw_alrp,wntw_alra,wntw_alr_npctl,wntw_riskv,wntw_risks
count,58.000000,58.0,58.000000,58.000000,5.800000e+01,5.800000e+01,5.800000e+01,58.000000,5.800000e+01,58.000000,...,58.000000,58.000000,58.000000,58.000000,5.800000e+01,5.800000e+01,5.800000e+01,58.000000,58.000000,58.000000
mean,216.500000,6.0,58.000000,6058.000000,6.810578e+05,1.189372e+11,8.929551e+08,2850.801356,3.558352e+08,90.590475,...,17133.508985,0.000281,17943.852688,17.845975,1.325291e-07,1.781265e-07,4.182517e-12,12.913158,21139.006898,15.866677
std,16.886879,0.0,33.773757,33.773757,1.462434e+06,2.297878e+11,1.444661e+09,3148.320814,7.742302e+08,14.121290,...,45551.522583,0.000549,46241.481408,19.771395,5.701245e-07,1.005443e-06,8.609047e-12,19.719188,53917.993628,19.833884
min,188.000000,6.0,1.000000,6001.000000,1.198000e+03,8.615997e+08,2.277225e+06,234.255013,3.031424e+06,23.926185,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,202.250000,6.0,29.500000,6029.500000,4.781175e+04,1.255870e+10,5.277571e+07,988.820072,4.040429e+07,89.102768,...,47.921519,0.000004,53.803418,4.681213,3.676700e-11,3.295500e-12,1.500000e-15,4.588363,46.952459,1.980592
50%,216.500000,6.0,58.000000,6058.000000,1.864480e+05,3.988174e+10,2.217376e+08,1611.629822,1.111793e+08,95.593382,...,4455.139340,0.000085,4798.944897,11.497988,3.915011e-09,7.507605e-10,1.690000e-13,5.277004,6302.901372,9.497296
75%,230.750000,6.0,86.500000,6086.500000,7.109165e+05,1.308460e+11,9.163897e+08,3871.159730,2.586308e+08,98.337576,...,10030.209221,0.000284,13336.708500,22.647787,2.459223e-08,1.585627e-08,2.890000e-12,10.174868,17681.866234,21.905822
max,245.000000,6.0,115.000000,6115.000000,1.000571e+07,1.517716e+12,6.586834e+09,20308.295400,5.326193e+09,100.000000,...,314669.624419,0.003148,317624.861271,88.796038,4.242441e-06,7.425749e-06,3.838500e-11,99.257196,370411.689955,88.514158


In [17]:
assert df3['oid_'].notnull().all(), "Missing oid_ values"
assert df3['statefips'].notnull().all(), "Missing statefips values"
assert df3['countyfips'].notnull().all(), "Missing countyfips values"

# Population and values should be non-negative
assert (df3['population'] >= 0).all(), "Negative population values found"
assert (df3['buildvalue'] >= 0).all(), "Negative buildvalue found"
assert (df3['agrivalue'] >= 0).all(), "Negative agrivalue found"

# Area should be positive (non-zero)
assert (df3['area'] > 0).all(), "Non-positive area values found"

# Risk scores and values should be within expected ranges
assert (df3['risk_value'] >= 0).all(), "Negative risk_value found"
assert (df3['risk_score'] >= 0).all(), "Negative risk_score found"
assert (df3['risk_score'] <= 100).all(), "Risk score above 100 found (assuming 0-100 scale)"

### Step 6: Create new CSV files after cleaning data

Turning each dataframe into a new CSV file

In [18]:
df1.to_csv('dataset1_cleaned.csv', index=False)

In [19]:
df2.to_csv('dataset2_cleaned.csv', index=False)

In [20]:
df3.to_csv('dataset3_cleaned.csv', index=False)